<a href="https://colab.research.google.com/github/joydeep03564/Machine_Learning-/blob/master/News_classification_Using_Distill_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive 
drive.mount('/content/drive')



Mounted at /content/drive


In [5]:
import pandas 
import numpy


In [2]:
!pip install git+https://github.com/huggingface/transformers.git


  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-1km5kp8w
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-1km5kp8w
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 9.1MB/s 
     |████████████████████████████████| 3.2MB 19.4MB/s 
  Created wheel for transformers: filename=transformers-4.5.0.dev0-cp37-none-any.whl size=1973510 sha256=6797dd1e664c0dadb4ed7f0b5a93beae301c05b9167427639555e9b4b054dd39
  Stored in directory: /tmp/pip-ephem-wheel-cache-_pnvuafb/wheels/33/eb/3b/4bf5dd835e865e472d4fc0754f35ac0edb08fe852e8f21655f
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=3801f0a7a527fe3bdecf8593325747731168b89426619d2c7b869471e9209a9d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123

In [3]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf
import pandas as pd

In [6]:
df = pd.read_csv('/content/drive/MyDrive/NLP_Projects/bbc-text.csv') # Change path to your download location
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [7]:
#See what categories there are
df['category'].unique()

array(['tech', 'business', 'sport', 'entertainment', 'politics'],
      dtype=object)

In [8]:
df['encoded_cat'] = df['category'].astype('category').cat.codes
df.head()

,category,text,encoded_cat
0,tech,tv future in the hands of viewers with home th...,4
1,business,worldcom boss left books alone former worldc...,0
2,sport,tigers wary of farrell gamble leicester say ...,3
3,sport,yeading face newcastle in fa cup premiership s...,3
4,entertainment,ocean s twelve raids box office ocean s twelve...,1


In [9]:
data_texts = df["text"].to_list() # Features (not-tokenized yet)
data_labels = df["encoded_cat"].to_list() # Lables

In [10]:
from sklearn.model_selection import train_test_split

# Split Train and Validation data
train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size=0.2, random_state=0)

# Keep some data for inference (testing)
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size=0.01, random_state=0)

In [11]:
# Using huggingface's DistilBERTToekniszer tokeniser: 

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [13]:
#Creating a Dataset object for Tensorflow
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

In [14]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [16]:
model.fit(train_dataset.shuffle(1000).batch(16), epochs=10, batch_size=16,
          validation_data=val_dataset.shuffle(1000).batch(16))

Epoch 1/10
111/111 [==============================] - 62s 563ms/step - loss: 0.0377 - accuracy: 0.9904 - val_loss: 0.1163 - val_accuracy: 0.9730
Epoch 2/10
111/111 [==============================] - 62s 563ms/step - loss: 0.0319 - accuracy: 0.9938 - val_loss: 0.0398 - val_accuracy: 0.9865
Epoch 3/10
111/111 [==============================] - 62s 563ms/step - loss: 0.0222 - accuracy: 0.9943 - val_loss: 0.0371 - val_accuracy: 0.9933
Epoch 4/10
111/111 [==============================] - 62s 563ms/step - loss: 0.0149 - accuracy: 0.9972 - val_loss: 0.0371 - val_accuracy: 0.9910
Epoch 5/10
111/111 [==============================] - 63s 564ms/step - loss: 0.0056 - accuracy: 0.9994 - val_loss: 0.0404 - val_accuracy: 0.9910
Epoch 6/10
111/111 [==============================] - 63s 563ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.0410 - val_accuracy: 0.9910
Epoch 7/10
111/111 [==============================] - 63s 565ms/step - loss: 8.6419e-04 - accuracy: 1.0000 - val_loss: 0.0409 - va